# **Installing packages**

In [ ]:
!pip install -qU pypdf
!pip install -qU langchain_community
!pip install -qU tiktoken
!pip install langgraph
!pip install -qU langchain-core
!pip install -qU langchain-huggingface

# **Chunk the document**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the PDF
loader = PyPDFLoader('/content/drive/MyDrive/Colab Notebooks/RAG.pdf')
pages = []

# Each page is a Document object
async for page in loader.alazy_load():
   pages.append(page)

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# Pass the list of Document objects
all_splits = text_splitter.split_documents(pages)

# Print first 5 chunks
print(all_splits[:5])


In [ ]:
# embedding model
!pip install -qU langchain-huggingface

#  **Create Embeddings**

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


# **Let’s create an In-memory VectorStore**



In [ ]:
# Create an In-Memory Vector Store
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)



# **Add the document chunks to the vector store**

In [ ]:
document_ids = vector_store.add_documents(documents=all_splits)

In [ ]:
# Install the Groq package
!pip install -qU "langchain[groq]"

# **Implement the Groq with llama-8b-8192 model**

In [ ]:
import getpass
import os

os.environ.get("GROQ_API_KEY")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

# **invoke the Chat to ask the questions from our PDF**

In [ ]:
from langchain import hub
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
   question: str
   context: List[Document]
   answer: str

# Define application steps
def retrieve(state: State):
   retrieved_docs = vector_store.similarity_search(state["question"])
   return {"context": retrieved_docs}

def generate(state: State):
    # Combine all context content
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])

    # Add system-level instruction
    system_message = {
        "role": "system",
        "content": (
            "You are a NIS AGENT named thidass. "
            "You perform actions, Guns, and can answer questions about your role. "
            "Always introduce yourself as thidass if someone asks your name."
        )
    }

    # Add the user question
    user_message = {
        "role": "user",
        "content": state["question"]
    }

    # Combine system instructions and user query
    messages = [system_message, {"role": "user", "content": docs_content}, user_message]

    # Generate a response
    response = llm.invoke(messages)

    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


# **ask questions**

In [ ]:
response = graph.invoke({"question": "What's your name?"})
print(response["answer"])
